In [1]:
import os
import numpy as np
import xarray as xr
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

# Load datasets for wind direction and speed from specified file paths
dir_data = xr.open_dataset('../../data/th.nc')
speed_data = xr.open_dataset('../../data/vs.nc')

# Extract longitude and latitude arrays from the datasets
lon = dir_data['lon'].values
lat = dir_data['lat'].values

# Define the number of arrows to be plotted in the quiver plot
num_arrows = 1000

# List of days to visualize (selected days covering specific storms)
day_list = [32, 33, 34, 35, 36, 87, 88, 89, 90, 91]
num_frames = len(day_list)

# Determine the step size for downsampling based on the number of arrows
# This ensures the arrows are spread across the grid but not too dense
step_lon = len(lon) // int(np.sqrt(num_arrows))
step_lat = len(lat) // int(np.sqrt(num_arrows))

# Create index arrays to sample the longitude and latitude based on the step size
lon_indices = np.arange(0, len(lon), step_lon)[:int(np.sqrt(num_arrows))]
lat_indices = np.arange(0, len(lat), step_lat)[:int(np.sqrt(num_arrows))]

# Create a mesh grid of sampled longitude and latitude values for plotting
sample_lon, sample_lat = np.meshgrid(lon[lon_indices], lat[lat_indices])
print("Running...")

# Function to generate images of wind direction analysis for each selected day
def generate_images():
    os.makedirs('../images/length_quiver', exist_ok=True)  # Ensure output directory exists

    for day in day_list:
        # Extract wind speed and direction for the specified day
        wind_speed = speed_data['wind_speed'].isel(day=day).values
        wind_dir = dir_data['wind_from_direction'].isel(day=day).values

        # Convert wind direction from degrees to radians for calculation
        wind_rad = np.deg2rad(wind_dir)

        # Calculate U and V components of wind vectors using sine and cosine
        # Note: U is the east-west component and V is the north-south component
        U = np.sin(wind_rad)  # U component
        V = np.cos(wind_rad)  # V component

        # Sample the U and V components and scale them by wind speed
        U_sample = U[lat_indices[:, None], lon_indices] * wind_speed[lat_indices[:, None], lon_indices]
        V_sample = V[lat_indices[:, None], lon_indices] * wind_speed[lat_indices[:, None], lon_indices]

        # Initialize the plot for the current day
        fig, ax = plt.subplots(figsize=(10, 7))
        
        # Create a Basemap projection for the region (USA and surroundings)
        m = Basemap(llcrnrlon=-123, llcrnrlat=20, urcrnrlon=-62, urcrnrlat=50,
                    projection='lcc', lat_1=33, lat_2=45, lat_0=39.5, lon_0=-98, ax=ax)

        # Draw the map boundaries and fill continents with specified colors
        m.drawmapboundary(fill_color='#A6CAE0')  # Light blue for water
        m.fillcontinents(color='#E1DCBD', lake_color='#A6CAE0', alpha=0.7)  # Tan for land
        m.drawcoastlines(color='#404040', linewidth=0.8)  # Coastline details
        m.drawcountries(color='#404040', linewidth=0.6)  # Country borders

        # Draw latitude and longitude grid lines
        m.drawparallels(np.arange(20, 51, 10), labels=[1, 0, 0, 0], fontsize=8, color='#808080', linewidth=0.5)
        m.drawmeridians(np.arange(-120, -60, 10), labels=[0, 0, 0, 1], fontsize=8, color='#808080', linewidth=0.5)

        # Convert sampled longitude and latitude to map projection coordinates
        x, y = m(sample_lon, sample_lat)

        # Create a quiver plot to visualize wind direction and speed
        q = m.quiver(x, y, U_sample, V_sample, color='blue', scale=150, width=0.0015)

        # Add a quiver key to indicate the meaning of arrow length
        plt.quiverkey(q, 0.9, 1.05, 4, '4 m/s', labelpos='E', coordinates='axes', fontproperties={'size': 10})

        # Set the title for the plot with the current day's information
        plt.title(f"Wind Direction Analysis - Day {day}\nArrow Length Indicates Wind Speed", fontsize=14, pad=20, fontweight='bold')
        plt.tight_layout()  # Adjust layout to prevent clipping

        # Save the figure as an image in the specified directory
        fig.savefig(f'../images/length_quiver/day_{day:02d}.png', format='png')
        plt.close(fig)  # Close the figure to free memory

# Function to compile saved images into a GIF animation
def create_gif():
    images = []
    for day in day_list:
        # Read each saved image and append it to the list for GIF creation
        images.append(imageio.imread(f'../images/length_quiver/day_{day:02d}.png'))
    
    # Save all frames into a single GIF file with a specified frame rate
    imageio.mimsave(f'../gifs/length_quiver_{num_frames}.gif', images, fps=2)

# Generate the images and create the GIF
generate_images()
create_gif()
print("Completed")

Running...
Completed
